In [2]:
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

In [3]:
train_path = os.path.join('./data/DATASETv7/TRAIN')
val_path = os.path.join('./data/DATASETv7/TEST')
img_size = 224
batch_size = 80

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
#     shear_range = 0.2,
#     rotation_range = 10,
#     zoom_range = 0.2,
#     horizontal_flip = True,
#     vertical_flip = True,
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (img_size, img_size),
    class_mode = 'categorical',
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size = (img_size, img_size),
    class_mode = 'categorical',
)

Found 1050 images belonging to 15 classes.
Found 300 images belonging to 15 classes.


In [5]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2

IMAGE_SHAPE = (224, 224)

feature_extractor = hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))

In [6]:
model = tf.keras.models.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(64, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(15, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              3540265   
                                                                 
 flatten (Flatten)           (None, 1001)              0         
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 64)                64128     
                                                                 
 dense_1 (Dense)             (None, 15)                975       
                                                                 
Total params: 3,605,368
Trainable params: 65,103
Non-trainable params: 3,540,265
_________________________________________________________________


In [9]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', 
              optimizer = Adam(), 
              metrics=['accuracy'])

In [10]:
# Train the model
history = model.fit(train_generator, 
                    epochs=20, 
                    validation_data = val_generator) 

Epoch 1/20
33/33 [==============================] - 26s 708ms/step - loss: 3.0460 - accuracy: 0.1943 - val_loss: 1.7936 - val_accuracy: 0.4567
Epoch 2/20
33/33 [==============================] - 23s 685ms/step - loss: 1.7078 - accuracy: 0.4543 - val_loss: 1.3745 - val_accuracy: 0.5900
Epoch 3/20
33/33 [==============================] - 22s 665ms/step - loss: 1.3166 - accuracy: 0.5867 - val_loss: 1.2178 - val_accuracy: 0.6333
Epoch 4/20
33/33 [==============================] - 21s 641ms/step - loss: 1.0705 - accuracy: 0.6457 - val_loss: 1.1120 - val_accuracy: 0.6500
Epoch 5/20
33/33 [==============================] - 21s 643ms/step - loss: 0.9144 - accuracy: 0.7057 - val_loss: 1.0798 - val_accuracy: 0.6433
Epoch 6/20
33/33 [==============================] - 21s 646ms/step - loss: 0.7705 - accuracy: 0.7457 - val_loss: 1.0147 - val_accuracy: 0.6600
Epoch 7/20
33/33 [==============================] - 21s 643ms/step - loss: 0.6954 - accuracy: 0.7752 - val_loss: 1.0191 - val_accuracy: 0.6633

In [11]:
history = model.fit(train_generator, 
                    epochs=20, 
                    validation_data = val_generator)

Epoch 1/20
33/33 [==============================] - 22s 678ms/step - loss: 0.1896 - accuracy: 0.9362 - val_loss: 1.1094 - val_accuracy: 0.7300
Epoch 2/20
33/33 [==============================] - 23s 699ms/step - loss: 0.1726 - accuracy: 0.9448 - val_loss: 1.0801 - val_accuracy: 0.7333
Epoch 3/20
33/33 [==============================] - 23s 706ms/step - loss: 0.1677 - accuracy: 0.9505 - val_loss: 1.0863 - val_accuracy: 0.7200
Epoch 4/20
33/33 [==============================] - 22s 683ms/step - loss: 0.1853 - accuracy: 0.9400 - val_loss: 1.1063 - val_accuracy: 0.7267
Epoch 5/20
33/33 [==============================] - 22s 669ms/step - loss: 0.1769 - accuracy: 0.9333 - val_loss: 1.0959 - val_accuracy: 0.7200
Epoch 6/20
33/33 [==============================] - 23s 685ms/step - loss: 0.1411 - accuracy: 0.9562 - val_loss: 1.0619 - val_accuracy: 0.7333
Epoch 7/20
33/33 [==============================] - 22s 676ms/step - loss: 0.1362 - accuracy: 0.9524 - val_loss: 1.1186 - val_accuracy: 0.7133